In [33]:
import cv2
import os

video_dirs = [
    r"f:\SKRIPSI\TA\DROZY\videos_i8"]

# Target FPS dan minimum frame
target_fps = 15
min_frames = 8000

selected_videos = []

# Iterasi melalui semua direktori
for video_dir in video_dirs:
    # Iterasi melalui semua file di direktori video
    for filename in os.listdir(video_dir):
        if filename.lower().endswith((".mov", ".mp4")):  # This handles both .MOV and .mov
            video_path = os.path.join(video_dir, filename)
            cap = cv2.VideoCapture(video_path)
            
            # Mendapatkan frame rate dan jumlah total frame
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            # Memeriksa apakah video memenuhi kriteria
            if fps >= target_fps and frame_count > min_frames:
                selected_videos.append((video_dir, filename))
            
            cap.release()

# Mencetak video yang memenuhi kriteria
print("Video yang memenuhi kriteria:")
for video_dir, video in selected_videos:
    print(f"{video} in {video_dir}")
    
print("Total video yang memenuhi kriteria:", len(selected_videos))


Video yang memenuhi kriteria:
1-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
1-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
1-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
11-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
11-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
11-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
14-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
14-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
14-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
2-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
2-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
2-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
3-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
3-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
3-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
4-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
4-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
4-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
5-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
5-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
5-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
6-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
6-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
6-3.mp4 in f:\SKRIPSI

FIX USE DLIB

In [37]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import csv
from imutils import face_utils

# Initialize dlib's face detector and load the facial landmark predictor
predictor_path = r"F:\SKRIPSI\TA\Kartivel\Driver-Drowsiness-Detection-main\shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])  # 65, 59
    B = dist.euclidean(mouth[12], mouth[16])  # 63, 57
    C = dist.euclidean(mouth[0], mouth[6])   # 49, 55
    return (A + B) / (2.0 * C)

def get_eye_image(frame, eye_points):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [eye_points], 255)
    eye = cv2.bitwise_and(frame, frame, mask=mask)
    (x, y, w, h) = cv2.boundingRect(eye_points)
    eye = eye[y:y+h, x:x+w]
    return eye

def process_eye_image(eye):
    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)
    _, binary_eye = cv2.threshold(blurred_eye, 30, 255, cv2.THRESH_BINARY_INV)
    return binary_eye

# Path to the video
video_path = r"F:\SKRIPSI\TA\DROZY\videos_i8\2-1.mp4"
cap = cv2.VideoCapture(video_path)
video_basename = os.path.basename(video_path)
video_title, _ = os.path.splitext(video_basename)
csv_filename = f"{video_title}_data_analysis.csv"

data_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 0)


    if faces:
        for face in faces:
            shape = predictor(gray, face)
            landmarks = face_utils.shape_to_np(shape)

            ear, mar = 0, 0     

            if len(landmarks) >= 48:  # Ensure there are enough points to compute mouth and eyes
                leftEye = landmarks[36:42]
                rightEye = landmarks[42:48]
                mouth = landmarks[48:68]

                ear_left = eye_aspect_ratio(leftEye)
                ear_right = eye_aspect_ratio(rightEye)
                ear = (ear_left + ear_right) / 2.0

                mar = mouth_aspect_ratio(mouth)

                left_eye_image = get_eye_image(frame, leftEye)
                right_eye_image = get_eye_image(frame, rightEye)

                left_eye_binary = process_eye_image(left_eye_image)
                right_eye_binary = process_eye_image(right_eye_image)

                cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)

                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                cv2.imshow("Left Eye Binary", left_eye_binary)
                cv2.imshow("Right Eye Binary", right_eye_binary)
            else:
                ear, mar = 0, 0  # Reset values if not enough landmarks

            cv2.imshow("Frame", frame)

            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            data_list.append([frame_number, ear, mar])
    else:
        # No faces detected
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        data_list.append([frame_number, 0, 0])  # Append zeros for this frame

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Writing data to CSV
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Frame', 'EAR', 'MAR'])
    csvwriter.writerows(data_list)

    print(f"Data written to {csv_filename}")

Data written to 2-1_data_analysis.csv


Auto RUN Video

In [43]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import csv
from imutils import face_utils
import os

# Initialize dlib's face detector and load the facial landmark predictor
predictor_path = r"F:\SKRIPSI\TA\Kartivel\Driver-Drowsiness-Detection-main\shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])  # 65, 59
    B = dist.euclidean(mouth[12], mouth[16])  # 63, 57
    C = dist.euclidean(mouth[0], mouth[6])   # 49, 55
    return (A + B) / (2.0 * C)

def process_videos_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):  # Check for video files
            video_path = os.path.join(folder_path, filename)
            process_video(video_path)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    video_title = os.path.splitext(os.path.basename(video_path))[0]
    csv_filename = f"{video_title}_data_analysis.csv"
    data_list = []


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if faces:
            for face in faces:
                shape = predictor(gray, face)
                landmarks = face_utils.shape_to_np(shape)

                ear, mar = 0, 0   

                if len(landmarks) >= 48:
                    leftEye = landmarks[36:42]
                    rightEye = landmarks[42:48]
                    mouth = landmarks[48:68]

                    ear = (eye_aspect_ratio(leftEye) + eye_aspect_ratio(rightEye)) / 2.0
                    mar = mouth_aspect_ratio(mouth)

                    cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
                    cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
                    cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)

                    cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    ear, mar = 0, 0 
                cv2.imshow("Frame", frame)

                frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                data_list.append([frame_number, ear, mar])

        else:
        # No faces detected
            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            data_list.append([frame_number, 0, 0])

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Writing data to CSV
    with open(csv_filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Frame', 'EAR', 'MAR'])
        csvwriter.writerows(data_list)
        print(f"Data written to {csv_filename}")

# Path to the directory containing videos
video_folder_path = r"F:\SKRIPSI\TA\DROZY\videos_i8\x"
process_videos_in_folder(video_folder_path)


Data written to 10-1_data_analysis.csv
Data written to 10-3_data_analysis.csv
Data written to 12-1_data_analysis.csv
Data written to 13-1_data_analysis.csv
Data written to 13-2_data_analysis.csv
Data written to 7-2_data_analysis.csv
Data written to 7-3_data_analysis.csv
Data written to 9-2_data_analysis.csv
Data written to 9-3_data_analysis.csv
